In [1]:
%pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
import sys
import csv

In [3]:
# Keys for authentication
consumer_key = "ONLQ0vLSAXM2tZDnckominEcK"
consumer_secret = "MpP7XurdGn1wpufX3rrfQgrs4AROnaw9ZiiZzKN6exr3zEDlN6"
access_token = "1274112117738737664-J2DjjoqqzN11CNp8bpnYzVp1dUYKM4"
access_token_secret = "3coYFJVCqHnOmD2m9Qt6zs7j1I24lhlPTU71eUOq4Zqg9"
bearer_token = "AAAAAAAAAAAAAAAAAAAAALJslQEAAAAA7EWdkTwrVDM44LIuewNascHjvoY%3DeXmhQyuXgo8zXJiSTMfqtmPA393HHO2W7nSrk0bK6b67SuJeb0"

In [4]:
# Authenticate
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret)

api = tweepy.API(auth)

In [5]:
def scrape_tweets(query: str, amount: int) -> list:
    """Scrapes a specified number of tweets based on given query and location.

    Method from https://www.sahilfruitwala.com/guide-to-extract-tweets-using-tweepy#how-to-retrieve-specific-number-of-tweets-using-tweepy 
    
    Args:
        query (str): The query
    Returns:
        list (str): List of contents of tweets.
    """
    extracted_tweets = []
    for tweet in tweepy.Cursor(api.search_tweets, query, count=100, tweet_mode="extended", result_type="mixed").items(amount):
        extracted_tweets.append(tweet.full_text)
    return extracted_tweets

In [6]:
def write_to_csv(tweets: list, csv_name: str) -> None:
    """Appends the list of tweets to csv.
    
    Largely copied from https://gist.github.com/anku255/0cebd75cce675f2b56de1ef48ec06575.

    """
    print("Writing {0} tweets to {1}".format(len(tweets), csv_name))
    tweets_for_csv = [[tweet.encode("utf-8")] for tweet in tweets]
    with open(csv_name, "a+") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerows(tweets_for_csv)


In [7]:
# queries: commented out becuase already scraped
# queries = [
#     ["(metaverse OR meta verse OR #metaverse OR #meta #verse) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(innovation OR #innovation OR innovate OR innovative OR #innovate OR #innovative) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(sustainability OR #sustainability OR sustainable) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(technology footprint OR technology OR #technologyfootprint) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(artificial intelligence OR ai OR #ai OR #artificialintelligence) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(ethics OR #ethics OR ethical OR #ethical) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(inflation OR #inflation) AND (place_country:AE OR uae) lang:en -is:retweet "],
#     ["(cop 28 OR cop28 OR #cop28 OR #cop #28) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(museum of the future OR @museumofthefuture OR #museumofthefuture OR #museum #of #the #future) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(bit coin OR bitcoin OR #bitcoin OR #crypto OR crpyto OR cryptocurrency) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(cyber attack OR #cyberattack OR #cyber OR #cyberattacks OR #cyber #attack) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(robot OR robots OR #robots OR #robot) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(drone OR dronesOR #drone OR #drones) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(hacker OR hacking OR #hacker OR #hacking OR #hack) AND (place_country:AE OR uae) lang:en -is:retweet"],
#     ["(chatgpt OR #chatgpt OR #chat #gpt OR chat gpt) AND (place_country:AE OR uae) lang:en -is:retweet"]
# ]

# queries = [["(nft OR #nft OR #nfts OR nfts) AND (place_country:AE OR uae) lang:en -is:retweet"]]


In [55]:
# just for testing
# query = "(innovation OR #innovation OR innovative) AND (place_country:AE OR uae) lang:en -is:retweet"
# tweets = scrape_tweets(query, 50)
# tweets

['Joined UAE Minister &amp; President-Designate for #COP28UAE HE Dr Sultan Al Jaber &amp; SG @IEA HE Mac Monigle at 9th Asian Ministerial Energy Roundtable to galvanise global efforts for “Mapping Stable &amp; Secure Energy pathways for achieving Energy security &amp; Justice, Growth &amp; Innovation” https://t.co/ZrQy31CrDz',
 'Foreign Secretary @AmbVMKwatra participated in a meeting of focal points of the India-France-UAE trilateral dialogue and discussed potential cooperation in the fields of defence, energy and environment, innovation and people-to-people exchanges. https://t.co/SpVNzPySNu',
 'As the UAE marks National Environment Day, we remain committed to advancing innovative solutions to benefit current and future generations. This crucial work will continue at COP28, as we invite the world to come together in pursuit of a more sustainable future. #UAEforClimate',
 'RT @KhalifaAlgaz: Did you know that the UAE is developing an Arabic ChatGPT using ~10 billion parameters?\n\nThe 

In [8]:
# Add the resulting tweets from the queries to the dataset
csv_name="dataset.csv"
total = 0
for query in queries:
    tweets = scrape_tweets(query=query, amount=250)
    total += len(tweets)
    print("Found {0} tweets related to the query {1}".format(len(tweets), query))
    write_to_csv(tweets=tweets,csv_name=csv_name)
print("Done. {0} rows.".format(total))

Found 250 tweets related to the query ['(nft OR #nft OR #nfts OR nfts) AND (place_country:AE OR uae) lang:en -is:retweet']
Writing 250 tweets to dataset.csv
Done. 250 rows.
